In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from collections import Counter
from nltk.stem import SnowballStemmer

In [38]:
dir = 'C:/_dados/_donors/'
stemmer = SnowballStemmer("english")

In [3]:
train = pd.read_csv(dir + 'train.csv', sep=',',index_col=0, dtype={11:'str',12:'str'})
test = pd.read_csv(dir + 'test.csv', sep=',',index_col=0, dtype={11:'str',12:'str'})
resources = pd.read_csv(dir + 'resources.csv', sep=',',index_col=0)

In [17]:
def _stemmer(text):
    s = text.split(' ')
    s2 = [stemmer.stem(str(x)) for x in s]
    return ' '.join(s2) 
print(_stemmer('hi cats dogs generously'))

def _cleaner(text):
    clean_line = text.upper()
    clean_line = re.sub("\\\\[RNT]", "", clean_line)
    clean_line = re.sub("[^A-Z ]+", "", clean_line)
    clean_line = re.sub("\s\s+", " ", clean_line)
    clean_line = re.sub("NANNAN", "", clean_line)
    return clean_line
print(_cleaner('jnij4r34 r34r 34 rfrvsdv 34f3rf3foro34f     r34fr3f3f34'))

hi cat dog generous
JNIJR RR RFRVSDV FRFFOROF RFRFF


In [9]:
train.groupby('project_subject_categories')['project_is_approved'].mean().sort_values(ascending = False)

project_subject_categories
Warmth_Care_Hunger                      0.921953
Health_Sports,Warmth_Care_Hunger        0.903226
History_Civics,Health_Sports            0.900000
Literacy_Language,History_Civics        0.890494
History_Civics,Literacy_Language        0.885490
Literacy_Language                       0.869858
Literacy_Language,Math_Science          0.867219
Music_The_Arts,Special_Needs            0.863830
Applied_Learning,Literacy_Language      0.861208
Health_Sports,Special_Needs             0.861064
Math_Science,Literacy_Language          0.861046
Literacy_Language,Applied_Learning      0.859345
Math_Science,History_Civics             0.858326
Literacy_Language,Special_Needs         0.854450
History_Civics,Math_Science             0.853333
Health_Sports,Applied_Learning          0.851852
Music_The_Arts                          0.850006
Health_Sports,Math_Science              0.848936
Health_Sports,Literacy_Language         0.847859
Health_Sports                           0.

In [5]:
train['project_subject_categories'] =  train['project_subject_categories'].str.replace('Warmth,','Warmth').str.replace(', ',',').str.replace('& ','').str.replace(' ','_')

In [6]:
train[['cat1','cat2']] = train['project_subject_categories'].str.split(',', expand=True)

In [7]:
resources['total'] = resources.quantity * resources.price
expenses = resources.groupby('id')['total'].sum()
expenses.head()

id
p000001     833.63
p000002     630.28
p000003     298.97
p000004    1126.22
p000005     702.31
Name: total, dtype: float64

In [10]:
number_of_items = resources.groupby('id')['total'].count()
number_of_items.head()

id
p000001     4
p000002    14
p000003     4
p000004    95
p000005     4
Name: total, dtype: int64

In [28]:
train2 = train.join(expenses)
train2 = train2.join(number_of_items,rsuffix='1')
train2.rename(columns={'total1':'qnty'}, inplace=True)
train2 = train2
train2.head()

teacher_id teacher_prefix school_state  \
id                                                                      
p036502  484aaf11257089a66cfedc9461c6bd0a            Ms.           NV   
p039565  df72a3ba8089423fa8a94be88060f6ed           Mrs.           GA   
p233823  a9b876a9252e08a55e3d894150f75ba3            Ms.           UT   
p185307  525fdbb6ec7f538a48beebaa0a51b24f            Mr.           NC   
p013780  a63b5547a7239eae4c1872670848e61a            Mr.           CA   

        project_submitted_datetime project_grade_category  \
id                                                          
p036502        2016-11-18 14:45:59          Grades PreK-2   
p039565        2017-04-26 15:57:28             Grades 3-5   
p233823        2017-01-01 22:57:44             Grades 3-5   
p185307        2016-08-12 15:42:11             Grades 3-5   
p013780        2016-08-06 09:09:11             Grades 6-8   

             project_subject_categories  \
id                                        
p036502               Literacy_Language   
p039565    Music_The_Arts,Health_Sports   
p233823  Math_Science,Literacy_Language   
p185307                   Health_Sports   
p013780                   Health_Sports   

                  project_subject_subcategories  \
id                                                
p036502                                Literacy   
p039565            Performing Arts, Team Sports   
p233823  Applied Sciences, Literature & Writing   
p185307                       Health & Wellness   
p013780                       Health & Wellness   

                                             project_title  \
id                                                           
p036502                           Super Sight Word Centers   
p039565                             Keep Calm and Dance On   
p233823                              Lets 3Doodle to Learn   
p185307  \"Kid Inspired\" Equipment to Increase Activit...   
p013780   We need clean water for our culinary arts class!   

                                           project_essay_1  \
id                                                           
p036502  Most of my kindergarten students come from low...   
p039565  Our elementary school is a culturally rich sch...   
p233823  Hello;\r\nMy name is Mrs. Brotherton. I teach ...   
p185307  My students are the greatest students but are ...   
p013780  My students are athletes and students who are ...   

                                           project_essay_2 project_essay_3  \
id                                                                           
p036502  I currently have a differentiated sight word c...             NaN   
p039565  We strive to provide our diverse population of...             NaN   
p233823  We are looking to add some 3Doodler to our cla...             NaN   
p185307  The student's project which is totally \"kid-i...             NaN   
p013780  For some reason in our kitchen the water comes...             NaN   

        project_essay_4                           project_resource_summary  \
id                                                                           
p036502             NaN  My students need 6 Ipod Nano's to create and d...   
p039565             NaN  My students need matching shirts to wear for d...   
p233823             NaN  My students need the 3doodler. We are an SEM s...   
p185307             NaN  My students need balls and other activity equi...   
p013780             NaN  My students need a water filtration system for...   

         teacher_number_of_previously_posted_projects  project_is_approved  \
id                                                                           
p036502                                            26                    1   
p039565                                             1                    0   
p233823                                             5                    1   
p185307                                            16                    0   

In [29]:
train2['full_text'] = train2['project_essay_1'].astype(str)+train2['project_essay_2'].astype(str)+train2['project_essay_3'].astype(str)+train2['project_essay_4'].astype(str)
train2['full_text'] = train2.full_text.map(_cleaner)
train2['stem'] = train2.full_text.map(_stemmer)
train2.head()

teacher_id teacher_prefix school_state  \
id                                                                      
p036502  484aaf11257089a66cfedc9461c6bd0a            Ms.           NV   
p039565  df72a3ba8089423fa8a94be88060f6ed           Mrs.           GA   
p233823  a9b876a9252e08a55e3d894150f75ba3            Ms.           UT   
p185307  525fdbb6ec7f538a48beebaa0a51b24f            Mr.           NC   
p013780  a63b5547a7239eae4c1872670848e61a            Mr.           CA   

        project_submitted_datetime project_grade_category  \
id                                                          
p036502        2016-11-18 14:45:59          Grades PreK-2   
p039565        2017-04-26 15:57:28             Grades 3-5   
p233823        2017-01-01 22:57:44             Grades 3-5   
p185307        2016-08-12 15:42:11             Grades 3-5   
p013780        2016-08-06 09:09:11             Grades 6-8   

             project_subject_categories  \
id                                        
p036502               Literacy_Language   
p039565    Music_The_Arts,Health_Sports   
p233823  Math_Science,Literacy_Language   
p185307                   Health_Sports   
p013780                   Health_Sports   

                  project_subject_subcategories  \
id                                                
p036502                                Literacy   
p039565            Performing Arts, Team Sports   
p233823  Applied Sciences, Literature & Writing   
p185307                       Health & Wellness   
p013780                       Health & Wellness   

                                             project_title  \
id                                                           
p036502                           Super Sight Word Centers   
p039565                             Keep Calm and Dance On   
p233823                              Lets 3Doodle to Learn   
p185307  \"Kid Inspired\" Equipment to Increase Activit...   
p013780   We need clean water for our culinary arts class!   

                                           project_essay_1  \
id                                                           
p036502  Most of my kindergarten students come from low...   
p039565  Our elementary school is a culturally rich sch...   
p233823  Hello;\r\nMy name is Mrs. Brotherton. I teach ...   
p185307  My students are the greatest students but are ...   
p013780  My students are athletes and students who are ...   

                                           project_essay_2  \
id                                                           
p036502  I currently have a differentiated sight word c...   
p039565  We strive to provide our diverse population of...   
p233823  We are looking to add some 3Doodler to our cla...   
p185307  The student's project which is totally \"kid-i...   
p013780  For some reason in our kitchen the water comes...   

                               ...                         project_essay_4  \
id                             ...                                           
p036502                        ...                                     NaN   
p039565                        ...                                     NaN   
p233823                        ...                                     NaN   
p185307                        ...                                     NaN   
p013780                        ...                                     NaN   

                                  project_resource_summary  \
id                                                           
p036502  My students need 6 Ipod Nano's to create and d...   
p039565  My students need matching shirts to wear for d...   
p233823  My students need the 3doodler. We are an SEM s...   
p185307  My students need balls and other activity equi...   
p013780  My students need a water filtration system for...   

        teacher_number_of_previously_posted_projects  project_is_approved  \
id                                                                 

In [30]:
groups = []
for x in train2.groupby('cat1')['cat1'].head(1):
    if x not in groups: groups.append(x)
groups

['Literacy_Language',
 'Music_The_Arts',
 'Math_Science',
 'Health_Sports',
 'Applied_Learning',
 'Special_Needs',
 'History_Civics',
 'Warmth_Care_Hunger']

In [31]:
text = train2.index +';'+ train2['project_is_approved'].astype(str) +';'+ train2['cat1'].astype(str) +';'+ train2['stem'].astype(str)

In [40]:
with open(dir+'/consolidated.txt', 'w') as t1:
    with open(dir+'/consolidated_text.txt', 'w') as t2:
        for line in text:
            try:
                idx,approved,g,line = line.split(';',3)
                t2.write(str(idx)+';'+str(approved)+';'+str(g)+';'+str(line) + '\n')
                con = Counter(line.split()).items()
                for item in con:
                    a,b = item
                    t1.write(str(idx)+';'+str(approved)+';'+str(g)+';'+str(a)+';' + str(b) + '\n')
            except:
                print('.', end='',flush=True)

In [47]:
consolidated = pd.read_csv(dir+'consolidated.txt',delimiter=';',names=['id','approved','group','stem','number'])
consolidated_text = pd.read_csv(dir+'consolidated_text.txt',delimiter=';',names=['id','approved','group','stem'])

In [ ]:
for group  in groups:
    with open(dir+'matriz_'+group+'.csv','w') as f:
        print(group)
        st = consolidated[consolidated.group == group]
        st2 = consolidated_text[consolidated_text.group == group]
        steemed = st.groupby(['stem','approved'],as_index = False)['number'].agg('sum').pivot('stem','approved').fillna(0)
        steemed['grade'] = steemed.min(axis=1) / steemed.max(axis=1)
        steemed = steemed[(steemed.max(axis=1) >= 10)]
        steemed = steemed.sort_values('grade')
        steemed = steemed[:5000]
        #display(st2.head())
        words = steemed.index.tolist()
        f.write('id;approved;'+';'.join(words) + '\n')

        for i,l in st2.iterrows():
            linha = str(l.id) + ';' + str(l.approved)
            v = l.stem.split(" ")
            for word in words:
                linha += ';'+ str((word in v)*1)
            f.write(linha + '\n')

Literacy_Language
